In [2]:
# !pip install gym

In [6]:
import gym
import numpy as np
import random
from gym import spaces
import pandas as pd

In [5]:
class MentalHealthEnv(gym.Env):
    def __init__(self, df):
        super(MentalHealthEnv, self).__init__()

        #loading the dataset
        self.df = df.copy()
        self.current_index = 0


        #define state: 7 mental health scores (ranging between 0 to 300)
        self.observation_space = spaces.Box(low = 0, high = 300, shape = (7,), dtype = np.float32)

        # defining action space
        self.actions = [
            "Engage in Outdoor activities",
            "Reduce the social media usage",
            "Watch educational videos",
            "Increase mindfulness practicess like yoga",
            "Interact with positive communities",
            "Avoid stress-inducing content",
            "Follow a structured daily routine"
        ]

        self.action_space = spaces.Discrete(len(self.actions))

        #initiating state: random values between 50 and 250 (to avoid zero-starting issues)
        self.state = np.random.uniform(low = 50, high = 250, size = (7,))

        # define action effects (each action affects specific scores)
        self.action_effects = {
            "Engage in Outdoor activities": np.array([10, 5, 0, 8, -5, 7, 0]),
            "Reduce the social media usage": np.array([5, -15, 0, 0, 10, 0, 0]),
            "Watch educational videos": np.array([5, 0, 15, 7, -3, 10, 0]),
            "Increase mindfulness practicess like yoga": np.array([12, 0, 0, 5, -8, 15, 0]),
            "Interact with positive communities": np.array([6, 0, -8, 0, 12, 0, 0]),
            "Avoid stress-inducing content": np.array([6, 0, -8, 0, 12, 0, 0]),
            "Follow a structured daily routine": np.array([10, 0, 0, 0, 7, 8, 0])
        }
        self.state = self.get_state_from_df()


    def get_state_from_df(self):
        # extract the initial state from the dataframe based on the current index
        row = self.df.iloc[self.current_index]
        return np.array(row[:-1])
    
    def step(self, action):
        #apply an action and update mental health scores

        self.state = np.clip(self.state)


        # reward: Improvement in mental health score 
        self.df.at[self.current_index, "Mental_Health_Score_DataFrame"] = np.sum(self.state)
        reward = self.df.at[self.current_index, "Mental_Health_Score_Dataframe"]

        done = np.all(self.state >= 280) #stops if all the score are near 300
        
        return self.state, reward, done, {}

    def reset(self):
        """Move to the next user in the dataset or restart if finished."""
        self.current_index += 1  # Move to the next row
        if self.current_index >= len(self.df):
            self.current_index = 0  # Reset if all users are processed
        self.state = self.get_state_from_df()
        return self.state

In [7]:
df = pd.read_csv("Mental_Health_Score_Dataframe.csv")

In [8]:
df.head()

,Emotional_Score,Social_Media_Time_Score,Platform_Score,Usage_Pattern_Score,Impact_Score,Additional_Feature_Score,TFIDF_Score,Mental_Health_Score
0,84,13,20,18,27,38,0,200
1,56,13,10,2,0,35,0,116
2,97,23,19,18,61,34,0,252
3,114,13,19,11,54,63,0,274
4,87,13,19,18,25,10,0,172


In [9]:
env = MentalHealthEnv(df)

### Model Training

In [12]:
from stable_baselines3 import DQN, A2C

In [11]:
!pip install stable-baselines3[extra]

  Using cached stable_baselines3-2.5.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached gymnasium-1.0.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached matplotlib-3.10.1-cp313-cp313-win_amd64.whl.metadata (11 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   --------------------- ------------------ 0.8/1.5 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 6.2 MB/s eta 0:00:00
Using cached gymnasium-1.0.0-py3-none-any.whl (958 kB)
   ----------------------------